# Fixed width Number

Although numeric types and the char type are distinct, Rust does provide byte literals, character-like literals for u8 values: `b'X'` represents the ASCII code for the character `X`, as a u8 value. For example, since the ASCII code for `A` is 65, the literals `b'A'` and 65u8 are exactly equivalent.

In [8]:
assert_eq!(b'A', 65u8);

Conversions that are out of range for the destination produce values that are equivalent to the original modulo 2^N, where N is the width of the destination in bits. This is sometimes called "truncation."

In [11]:
assert_eq!(1000_i16 as u8, 232_u8);
assert_eq!(65535_u32 as i16, -1_i16);

assert_eq!( -1_i8 as u8, 255_u8); 
assert_eq!( 255_u8 as i8, -1_i8);

The standard library provides some operations as methods on integers. For example:

In [12]:
assert_eq!(2_u16.pow(4), 16); // exponentiation 
assert_eq!((-4_i32).abs(), 4); // absolute value 
assert_eq!(0b101101_u8.count_ones(), 4); // population count

In real code, you usually won’t need to write out the type suffixes as we’ve done here, because the context will determine the type. When it doesn’t, however, the error mes‐ sages can be surprising. For example, the following doesn’t compile:

In [13]:
println!("{}", (-4).abs());

Error: can't call method `abs` on ambiguous numeric type `{integer}`

Rust wants to know exactly which integer type a value has before it will call the type’s own methods. The default of `i32` applies only if the type is still ambiguous after all method calls have been resolved, so that’s too late to help here. The solution is to spell out which type you intend, either with a suffix or by using a specific type’s function:

In [16]:
println!("{}", (-4_i32).abs());
println!("{}", -4_i32.abs());
println!("{}", -(4_i32.abs()));
println!("{}", i32::abs(-4));

4
-4
-4
4


In [18]:
let mut i = 1; loop {
    i *= 10; // panic: attempt to multiply with overflow // (but only in debug builds!)
}

thread '<unnamed>' panicked at 'attempt to multiply with overflow', src/lib.rs:113:5
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::panicking::panic
   3: <core::panic::unwind_safe::AssertUnwindSafe<F> as core::ops::function::FnOnce<()>>::call_once
   4: _run_user_code_10
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


In [19]:
let mut i:i32 = 1; 
loop {
    // panic: multiplication overflowed (in any build)
    i = i.checked_mul(10).expect("multiplication overflowed");
}

thread '<unnamed>' panicked at 'multiplication overflowed', src/lib.rs:114:27
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::option::expect_failed
   3: <unknown>
   4: <unknown>
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


In [22]:
assert_eq!(127_i8+1, -128);

Error: this arithmetic operation will overflow

Wrapping operations return the value equivalent to the mathematically correct result modulo the range of the value:

In [77]:
// The first product can be represented as a u16; 
// the second cannot, so we get 250000 modulo 216. 
assert_eq!(100_u16.wrapping_mul(200), 20000); 
assert_eq!(500_u16.wrapping_mul(500), 53392);
// Operations on signed types may wrap to negative values.
assert_eq!(500_i16.wrapping_mul(500), -12144);
// In bitwise shift operations, the shift distance
// is wrapped to fall within the size of the value.
// So a shift of 17 bits in a 16-bit type is a shift
// of 1.
assert_eq!(5_i16.wrapping_shl(17), 10);

Saturating operations return the representable value that is closest to the mathematically correct result. In other words, the result is “clamped” to the maximum and minimum values the type can represent:

In [78]:
assert_eq!(32760_i16.saturating_add(10), 32767); 
assert_eq!((-32760_i16).saturating_sub(10), -32768);

Overflowing operations return a tuple `(result, overflowed)`, where result is what the wrapping version of the function would return, and overflowed is a bool indicating whether an overflow occurred:

In [79]:
assert_eq!(255_u8.overflowing_sub(2), (253, false)); 
assert_eq!(255_u8.overflowing_add(2), (1, true));

`overflowing_shl` and `overflowing_shr` deviate from the pattern a bit: they return true for overflowed only if the shift distance was as large or larger than the bit width of the type itself. The actual shift applied is the requested shift mod‐ ulo the bit width of the type:

In [27]:
// A shift of 17 bits is too large for `u16`, and 17 modulo 16 is 1.
assert_eq!(5_u16.overflowing_shl(17), (10, true));

## float
max and inf

In [34]:
println!("{}", i32::MAX);
println!("{}", i64::MAX);
println!("{}", f32::MAX);
println!("{}", f32::INFINITY);
println!("{}", f32::NEG_INFINITY);

2147483647
9223372036854775807
340282350000000000000000000000000000000
inf
-inf


In [35]:
assert_eq!(5f32.sqrt() * 5f32.sqrt(), 5.); // exactly 5.0, per IEEE 
assert_eq!((-1.01f64).floor(), -2.0);

Again, method calls have a higher precedence than prefix operators, so be sure to correctly parenthesize method calls on negated values.

The `std::f32::consts` and `std::f64::consts` modules provide various commonly used mathematical constants like `E`, `PI`, and the square root of two.

When searching the documentation, remember that there pages for both the types themselves, named “f32 (primitive type)” and “f64 (primitive type), and the modules for each type, `std::f32` and `std::f64`.

As with integers, you usually won’t need to write out type suffixes on floating-point literals in real code, but when you do, putting a type on either the literal or the func‐ tion will suffice:

In [36]:
println!("{}", (2.0_f64).sqrt()); 
println!("{}", f64::sqrt(2.0));

1.4142135623730951
1.4142135623730951


Unlike C and C++, Rust performs almost no numeric conversions implicitly. If a function expects an `f64` argument, it’s an error to pass an `i32` value as the argument. In fact, Rust won’t even implicitly convert an `i16` value to an `i32` value, even though every `i16` value is also an `i32` value. But you can always write out explicit conversions using the as operator: `i as f64`, or `x as i32`.

In [38]:
// bool in rust
assert_eq!(false as i32, 0);
assert_eq!(true as i32, 1);

## char
you can write out a character’s Unicode code point in hexadecimal

In [47]:
println!("{}", '\x2A');
println!("{}", '\u{2A}');
println!("{}", '\u{10ffff}');

*
*
􏿿


Going in the other direction, `u8` is the only type the as operator will convert to char: Rust intends the as operator to perform only cheap, infallible conversions, but every integer type other than `u8` includes values that are not permitted Unicode code points, so those conversions would require run-time checks. Instead, the standard library function `std::char::from_u32` takes any `u32` value and returns an `Option<char>`: if the `u32` is not a permitted Unicode code point, then `from_u32` returns None; otherwise, it returns `Some(c)`, where `c` is the char result.

The standard library provides some useful methods on characters, which you can look up in the online documentation under “char (primitive type),” and the module “std::char.” For example:

In [51]:
assert_eq!('*'.is_alphabetic(), false); 
assert_eq!('β'.is_alphabetic(), true); 
assert_eq!('8'.to_digit(10), Some(8)); 
assert_eq!('ಠ'.len_utf8(), 3); 
assert_eq!(std::char::from_digit(2, 10), Some('2'));
assert_eq!(std::char::from_digit(15, 16), Some('f'));

## tuples
A tuple is a pair, or triple, quadruple, quintuple, etc. `(hence, n-tuple, or tuple)`, of val‐ ues of assorted types. You can write a tuple as a sequence of elements, separated by commas and surrounded by parentheses. For example, `("Brazil", 1985)` is a tuple whose first element is a statically allocated string, and whose second is an integer; its type is `(&str, i32)`. Given a tuple value t, you can access its elements as `t.0`, `t.1`, and so on.

In [55]:
// fn split_at(&self, mid: usize) -> (&str, &str);
let text = "I see the eigenvalue in thine eye"; 
let (head, tail): (&str, &str) = text.split_at(21); 
assert_eq!(head, "I see the eigenvalue "); 
assert_eq!(tail, "in thine eye");

In [61]:
// &'static str live through the program
let text = "I see the eigenvalue in thine eye"; 
let temp: (&'static str, &'static str) = text.split_at(21);
let head: &'static str = temp.0;
let tail: &'static str  = temp.1;
assert_eq!(head, "I see the eigenvalue ");
assert_eq!(tail, "in thine eye");

You’ll also see tuples used as a sort of minimal-drama struct type. For example, in the Mandelbrot program in Chapter 2, we needed to pass the width and height of the image to the functions that plot it and write it to disk. We could declare a struct with width and height members, but that’s pretty heavy notation for something so obvi‐ ous, so we just used a tuple:

In [62]:
/// Write the buffer `pixels`, whose dimensions are given by `bounds`, to the /// file named `filename`.
fn write_image(filename: &str, pixels: &[u8], bounds: (usize, usize))
-> Result<(), std::io::Error> {Ok(())}

The type of the bounds parameter is `(usize, usize)`, a tuple of two `usize` values. Admittedly, we could just as well write out separate width and height parameters, and the machine code would be about the same either way. It’s a matter of clarity. We think of the size as one value, not two, and using a tuple lets us write what we mean.

The other commonly used tuple type is the zero-tuple `()`. This is traditionally called the unit type because it has only one value, also written `()`. Rust uses the unit type where there’s no meaningful value to carry, but context requires some sort of type nonetheless.

In [64]:
// fn swap<T>(x: &mut T, y: &mut T);
// fn swap<T>(x: &mut T, y: &mut T) -> ();

For consistency’s sake, there are even tuples that contain a single value. The literal `("lonely hearts",)` is a tuple containing a single string; its type is `(&str,)`. Here, the comma after the value is necessary to distinguish the singleton tuple from a sim‐ ple parenthetic expression.

# Pointer Types
This is a big difference between Rust and most languages with garbage collection. In Java, if class Rectangle contains a field Vector2D upperLeft;, then upperLeft is a reference to another separately created Vector2D object. Objects never physically contain other objects in Java.

**Rust is different. The language is designed to help keep allocations to a minimum. Values nest by default. The value `((0, 0), (1440, 900))` is stored as four adjacent integers. If you store it in a local variable, you’ve got a local variable four integers wide. Nothing is allocated in the heap.**

This is great for memory efficiency, but as a consequence, when a Rust program needs values to point to other values, it must use pointer types explicitly. The good news is that the pointer types used in safe Rust are constrained to eliminate undefined behavior, so pointers are much easier to use correctly in Rust than in C++.

## References
At run time, a reference to an `i32` is a single machine word holding the address of the `i32`, which may be on the stack or in the heap. The expression `&x` produces a reference to `x`; in Rust terminology, we say that it borrows a reference to `x`. Given a reference `r`, the expression `*r` refers to the value `r` points to. These are very much like the `&` and `*` operators in C and C++. And like a C pointer, a reference does not automatically free any resources when it goes out of scope.

Unlike C pointers, however, Rust references are never null: there is simply no way to produce a null reference in safe Rust. And unlike C, Rust tracks the ownership and lifetimes of values, so mistakes like dangling pointers, double frees, and pointer inva‐ lidation are ruled out at compile time.

Rust references come in two flavors: 

`&T`: 
An immutable, shared reference. You can have many shared references to a given value at a time, but they are read-only: modifying the value they point to is for‐ bidden, as with `const T*` in C.

`&mut T`: 
A mutable, exclusive reference. You can read and modify the value it points to, as with a T* in C. But for as long as the reference exists, you may not have any other references of any kind to that value. In fact, the only way you may access the value at all is through the mutable reference.

**Rust uses this dichotomy between shared and mutable references to enforce a “single writer or multiple readers” rule: either you can read and write the value, or it can be shared by any number of readers, but never both at the same time. This separation, enforced by compile-time checks, is central to Rust’s safety guarantees. Chapter 5 explains Rust’s rules for safe reference use.**

## Boxes
The simplest way to allocate a value in the heap is to use `Box::new:`

In [67]:
let t = (12, "eggs");
let b: Box<(i32, &str)> = Box::new(t); // allocate a tuple in the heap

## Raw Pointers
Rust also has the raw pointer types *mut T and *const T. Raw pointers really are just like pointers in C++. Using a raw pointer is unsafe, because Rust makes no effort to track what it points to. For example, raw pointers may be null, or they may point to memory that has been freed or that now contains a value of a different type. All the classic pointer mistakes of C++ are offered for your enjoyment.

However, you may only dereference raw pointers within an unsafe block. An unsafe block is Rust’s opt-in mechanism for advanced language features whose safety is up to you. If your code has no unsafe blocks (or if those it does are written correctly), then the safety guarantees we emphasize throughout this book still hold. For details, see Chapter 22.

## Arrays, Vectors, and Slices
Rust has three types for representing a sequence of values in memory:
* The type `[T; N]` represents an array of `N` values, each of type `T`. An array’s size is a constant determined at compile time and is part of the type; you can’t append new elements or shrink an array.
* The type `Vec<T>`, called a vector of Ts, is a dynamically allocated, growable sequence of values of type `T`. A vector’s elements live on the heap, so you can resize vectors at will: push new elements onto them, append other vectors to them, delete elements, and so on.
* The types `&[T]` and `&mut [T]`, called a shared slice of Ts and mutable slice of Ts, are references to a series of elements that are a part of some other value, like an array or vector. You can think of a slice as a pointer to its first element, together with a count of the number of elements you can access starting at that point. A mutable slice `&mut [T]` lets you read and modify elements, but can’t be shared; a shared slice `&[T]` lets you share access among several readers, but doesn’t let you modify elements.

Given a value `v` of any of these three types, the expression `v.len()` gives the number of elements in `v`, and `v[i]` refers to the ith element of `v`. The first element is `v[0]`, and the last element is `v[v.len() - 1]`. Rust checks that `i` always falls within this range; if it doesn’t, the expression panics. The length of `v` may be zero, in which case any attempt to index it will panic. `i` must be a usize value; you can’t use any other integer type as an index.

### Array
There are several ways to write array values. The simplest is to write a series of values within square brackets:

In [68]:
let lazy_caterer: [u32; 6] = [1, 2, 4, 7, 11, 16]; 
let taxonomy = ["Animalia", "Arthropoda", "Insecta"];
assert_eq!(lazy_caterer[3], 7); 
assert_eq!(taxonomy.len(), 3);

For the common case of a long array filled with some value, you can write [V; N], where V is the value each element should have, and N is the length.

let mut sieve = [true; 10000]; 
for i in 2..100 {
    if sieve[i] { 
        let mut j=i*i; 
        while j < 10000 {
            sieve[j] = false;
            j+=i; 
        }
    } 
}
assert!(sieve[211]); 
assert!(!sieve[9876]);

You’ll see this syntax used for fixed-size buffers: `[0u8; 1024]` can be a one-kilobyte buffer, filled with zeros. Rust has no notation for an uninitialized array. (In general, Rust ensures that code can never access any sort of uninitialized value.)

An array’s length is part of its type and fixed at compile time. If n is a variable, you can’t write `[true; n]` to get an array of n elements. When you need an array whose length varies at run time (and you usually do), use a vector instead.

The useful methods you’d like to see on arrays—iterating over elements, searching, sorting, filling, filtering, and so on—are all provided as methods on slices, not arrays. But Rust implicitly converts a reference to an array to a slice when searching for methods, so you can call any slice method on an array directly:

In [71]:
let mut chaos = [3,5,4,1,2]; 
chaos.sort();
assert_eq!(chaos, [1, 2, 3, 4, 5]);

Here, the sort method is actually defined on slices, but since it takes its operand by reference, 
Rust implicitly produces a `&mut [i32]` slice referring to the entire array and passes that to sort to operate on. 
In fact, the `len` method we mentioned earlier is a slice method as well.

### Vectors
A vector `Vec<T>` is a resizable array of elements of type `T`, **allocated on the heap**.

There are several ways to create vectors. The simplest is to use the vec! macro, which
gives us a syntax for vectors that looks very much like an array literal:

In [80]:
let mut primes: Vec<i32> = vec![2, 3, 5, 7]; 
assert_eq!(primes.iter().product::<i32>(), 210);

But of course, this is a vector, not an array, so we can add elements to it dynamically:

In [81]:
primes.push(11);
primes.push(13); 
assert_eq!(primes.iter().product::<i32>(), 30030);

You can also build a vector by repeating a given value a certain number of times, again using a syntax that imitates array literals:

In [82]:
fn new_pixel_buffer(rows: usize, cols: usize) -> Vec<u8> { 
    vec![0; rows * cols]
}

The `vec!` macro is equivalent to calling `Vec::new` to create a new, empty vector and then pushing the elements onto it, which is another idiom:


In [86]:
let mut pal: Vec<&str> = Vec::new();
pal.push("step");
pal.push("on");
pal.push("no");
pal.push("pets");
assert_eq!(pal, vec!["step", "on", "no", "pets"]);

Another possibility is to build a vector from the values produced by an iterator:

In [87]:
let v: Vec<i32> = (0..5).collect(); 
assert_eq!(v, [0, 1, 2, 3, 4]);

As with arrays, you can use slice methods on vectors:

In [89]:
// A palindrome!
let mut palindrome: Vec<&str> = vec!["a man", "a plan", "a canal", "panama"]; palindrome.reverse();
// Reasonable yet disappointing:
assert_eq!(palindrome, vec!["panama", "a canal", "a plan", "a man"]);

Here, the `reverse` method is actually defined on slices, but the call implicitly borrows a `&mut [&str]` slice from the vector and invokes reverse on that.

A `Vec<T>` consists of three values: a pointer to the heap-allocated buffer for the ele‐ ments, which is created and owned by the `Vec<T>`; the number of elements that buffer has the capacity to store; and the number it actually contains now (in other words, its length).

If you know the number of elements a vector will need in advance, instead of `Vec::new` you can call `Vec::with_capacity` to create a vector with a buffer large enough to hold them all, right from the start; then, you can add the elements to the vector one at a time without causing any reallocation. The `vec!` macro uses a trick like this, since it knows how many elements the final vector will have. Note that this only establishes the vector’s initial size; if you exceed your estimate, the vector simply enlarges its storage as usual.

Many library functions look for the opportunity to use `Vec::with_capacity` instead of `Vec::new`. For example, in the collect example, the iterator `0..5` knows in advance that it will yield five values, and the collect function takes advantage of this to pre-allocate the vector it returns with the correct capacity. 

Just as a vector’s len method returns the number of elements it contains now, its capacity method returns the number of elements it could hold without reallocation:

In [92]:
let mut v: Vec<i32> = Vec::with_capacity(2); 
assert_eq!(v.len(), 0); 
assert_eq!(v.capacity(), 2);
v.push(1);
v.push(2);
assert_eq!(v.len(), 2);
assert_eq!(v.capacity(), 2);
v.push(3);
assert_eq!(v.len(), 3);
// Typically prints "capacity is now 4": 
println!("capacity is now {}", v.capacity());

capacity is now 4


You can insert and remove elements wherever you like in a vector, although these operations shift all the elements after the affected position forward or backward, so they may be slow if the vector is long:

In [94]:
let mut v: Vec<i32> = vec![10, 20, 30, 40, 50];
// Make the element at index 3 be 35.
v.insert(3, 35);
assert_eq!(v, [10, 20, 30, 35, 40, 50]);
// Remove the element at index 1.
v.remove(1);
assert_eq!(v, [10, 30, 35, 40, 50]);

You can use the `pop` method to remove the last element and return it. More precisely, popping a value from a `Vec<T>` returns an `Option<T>`: None if the vector was already empty, or `Some(v)` if its last element had been `v`:

In [96]:
let mut v: Vec<&str> = vec!["Snow Puff", "Glass Gem"]; 
assert_eq!(v.pop(), Some("Glass Gem")); 
assert_eq!(v.pop(), Some("Snow Puff")); 
assert_eq!(v.pop(), None);

You can use a `for` loop to iterate over a vector:

In [97]:
  // Get our command-line arguments as a vector of Strings.
let languages: Vec<String> = std::env::args().skip(1).collect(); 
for l in languages {
    println!("{}: {}", l, 
            if l.len()%2==0 {
                "functional"
            } else { 
                "imperative"
            }
    );
}

()

### Slices
A slice, written `[T]` without specifying the length, is a region of an array or vector. Since a slice can be any length, slices can’t be stored directly in variables or passed as function arguments. **Slices are always passed by reference**.

A reference to a slice is a fat pointer: a two-word value comprising a pointer to the slice’s first element, and the number of elements in the slice.

Suppose you run the following code:

In [15]:
let v: Vec<f64> = vec![0.0, 0.707, 1.0, 0.707]; 
let w: [f64; 4] = [0.0, -0.707, -1.0, -0.707];
{
    // v 是一个 Vec<f64>，它是动态分配的，具有动态生命周期，当 v 被创建时，
    // 它的生命周期开始，当它超出作用域时，它的生命周期结束。
    // w 是一个 [f64; 4]，它是一个栈上分配的数组，具有静态生命周期 'static，
    // 因为它的生命周期在整个程序运行期间都是有效的。

    // 因此，当你创建 sv 和 sw 时，它们的生命周期也会继承它们所引用的数据的生命周期。
    // sv 的生命周期会与 v 的生命周期一致，而 sw 的生命周期会与 w 的生命周期一致。

    // 由于生命周期不同，Rust 不允许你在同一作用域中将具有不同生命周期的引用传递给函数或宏。
    // 这是为了确保引用的安全性。要解决这个问题，你可以将 sv 和 sw 放在不同的作用域中，
    // 或者使用它们的值而不是引用，以便它们的生命周期不会重叠。
    let sv: &[f64] = &v;
    let sw: &[f64] = &w;
    println!("{}, {}", sv[3], sw[3]);
}; // sv 和 sw 的生命周期结束
// let sv: &[f64] = &v; 
// let sw: &[f64] = &w;
// println!("{}, {}", sv[3], sw[3]);

fn print(n: &[f64]) { 
    for elt in n{
        println!("{}", elt);
    }
}
print(&w); // works on arrays
print(&v); // works on vectors

0.707, -0.707
0
-0.707
-1
-0.707
0
0.707
1
0.707


# String Types

### String literal

In [20]:
println!("In the room the women come and go,
        Singing of Mount Abora");

In the room the women come and go,
        Singing of Mount Abora


In [21]:
println!("In the room the women come and go,
Singing of Mount Abora");

In the room the women come and go,
Singing of Mount Abora


In [22]:
println!("It was a bright, cold day in April, and \
        there were four of us—\
        more or less.");


It was a bright, cold day in April, and there were four of us—more or less.


In [23]:
println!("It was a bright, cold day in April, and \
                there were four of us—\
                more or less.");

It was a bright, cold day in April, and there were four of us—more or less.


In [29]:
println!(r"C:\Program Files\Gorillas"); 
// let pattern = Regex::new(r"\d+(\.\d+)*");
println!(r"\d+(\.\d+)*"); 

C:\Program Files\Gorillas
\d+(\.\d+)*


In [30]:
println!(r###"
        This raw string started with 'r###"'.
        Therefore it does not end until we reach a quote mark ('"')
        followed immediately by three pound signs ('###'):
"###);


        This raw string started with 'r###"'.
        Therefore it does not end until we reach a quote mark ('"')
        followed immediately by three pound signs ('###'):



In [31]:
println!(r###"
This raw string started with 'r###"'.
        Therefore it does not end until we reach a quote mark ('"')
        followed immediately by three pound signs ('###'):
"###);


This raw string started with 'r###"'.
        Therefore it does not end until we reach a quote mark ('"')
        followed immediately by three pound signs ('###'):



### Byte Strings
A string literal with the b prefix is a byte string. Such a string is a slice of `u8` values that is, bytes—rather than Unicode text:

In [32]:
let method = b"GET";
assert_eq!(method, &[b'G', b'E', b'T']);

The type of method is `&[u8; 3]`: it’s a reference to an array of three bytes. It doesn’t have any of the string methods we’ll discuss in a minute. The most string-like thing about it is the syntax we used to write it.

### String in Memory
A `String` or `&str`’s `.len()` method returns its length. The length is measured in bytes, not characters:

In [33]:
assert_eq!("ಠ_ಠ".len(), 7); 
assert_eq!("ಠ_ಠ".chars().count(), 3);

The type `&mut str` does exist, but it is not very useful, since almost any operation on UTF-8 can change its overall byte length, and a slice cannot reallocate its referent. In fact, the only operations available on `&mut str` are `make_ascii_uppercase` and `make_ascii_lowercase`, which modify the text in place and affect only single-byte characters, by definition.

### String
`&str` is very much like `&[T]`: a fat pointer to some data. `String` is analogous to `Vec<T>`.

Like a `Vec`, each `String` has its own heap-allocated buffer that isn’t shared with any other `String`. When a `String` variable goes out of scope, the buffer is automatically freed, unless the `String` was moved.

In [35]:
let error_message: String = "too many pets".to_string();
println!("{}", error_message);

too many pets


In [36]:
let findmyway: String = format!("{}°{:02}′{:02}′′N", 24, 5, 23);
println!("{}", findmyway);

24°05′23′′N


In [38]:
let bits: Vec<&str> = vec!["veni", "vidi", "vici"]; 
assert_eq!(bits.concat(), "venividivici"); 
assert_eq!(bits.join(", "), "veni, vidi, vici");

### Using Strings
Strings support the `==` and `!=` operators. Two strings are equal if they contain the same characters in the same order (regardless of whether they point to the same loca‐ tion in memory):

In [39]:
 assert!("ONE".to_lowercase() == "one");

Strings also support the comparison operators `<`, `<=`, `>`, and `>=`, as well as many useful methods and functions that you can find in the online documentation under “str (primitive type)” or the “std::str” module

In [41]:
assert!("peanut".contains("nut")); 
assert_eq!("ಠ_ಠ".replace("ಠ", "■"), "■_■"); 
assert_eq!(" clean\n".trim(), "clean");
for word in "veni, vidi, vici".split(", ") { 
    assert!(word.starts_with("v"));
};

Keep in mind that, given the nature of Unicode, simple char-by-char comparison does not always give the expected answers. For example, the Rust strings "th\u{e9}" and "the\u{301}" are both valid Unicode representations for thé, the French word for tea. Unicode says they should both be displayed and processed in the same way, but Rust treats them as two completely distinct strings. Similarly, Rust’s ordering operators like `<` use a simple lexicographical order based on character code point values. This ordering only sometimes resembles the ordering used for text in the user’s language and culture. 

### Other String-Like Types
Rust guarantees that strings are valid UTF-8. Sometimes a program really needs to be able to deal with strings that are not valid Unicode. This usually happens when a Rust program has to interoperate with some other system that doesn’t enforce any such rules. For example, in most operating systems it’s easy to create a file with a filename that isn’t valid Unicode. What should happen when a Rust program comes across this sort of filename?
Rust’s solution is to offer a few string-like types for these situations:
* Stick to String and &str for Unicode text.
* When working with filenames, use `std::path::PathBuf` and `&Path` instead.
* When working with binary data that isn’t UTF-8 encoded at all, use `Vec<u8>` and `&[u8]`.
* When working with environment variable names and command-line arguments in the native form presented by the operating system, use `OsString` and `&OsStr`.
* When interoperating with C libraries that use null-terminated strings, use `std::ffi::CString` and `&CStr`.

## Type Aliases
The type keyword can be used like typedef in C++ to declare a new name for an existing type:

In [42]:
type Bytes = Vec<u8>;

The type Bytes that we’re declaring here is shorthand for this particular kind of Vec:

In [44]:
fn decode(data: &Bytes) { 
    //...
}